In [1]:
from numpy.random import RandomState
import numpy as np
import qctoolkit as qtk

In [2]:
maia = {
    #'flags': '-l runtime=6:0:0',#membycore=4G',
    'ip': '131.152.25.38',
    'submission_script': '$HOME/bin/qsubqtk.sh',
    'username': 'changk',
    'compress': True,
}

qmsetting = {
    'program': 'gaussian',
    'theory': 'pbe0',
    'vdw': 'd3',
    #'theory': 'ccsd(t)',
    #'basis_set': 'aug-cc-pVTZ',
    'openshell': 'unrestricted',
    'threads': 1,
    'save_density': True,
    'print_energy': True,
    #'charge_multiplicity': [0, 1]
}

In [3]:
covalentList = np.arange(0.5, 2.0, 0.08)
hbondList = np.arange(0.7, 4.0, 0.16)
HF_covalent = 0.93
HCl_covalent = 1.27
HBr_covalent = 1.41
HI_covalent = 1.61
HF_hbond = 1.88
cbonds = [HF_covalent, HCl_covalent, HBr_covalent, HI_covalent]
hbonds = [2.0, 3.0, 3.5, 3.5]

cbond_list = []
for cb in cbonds:
    cbl = np.round(np.linspace(0.5*cb, 2*cb, 19), decimals=2).tolist()
    cbond_list.append(cbl)
    
hbond_list = []
for hb in hbonds:
    hbl = np.round(np.linspace(0.5*hb, 2*hb, 19), decimals=2).tolist()
    hbond_list.append(hbl)

print hbond_list
halogen = [9, 17, 35, 53]

[[1.0, 1.17, 1.33, 1.5, 1.67, 1.83, 2.0, 2.17, 2.33, 2.5, 2.67, 2.83, 3.0, 3.17, 3.33, 3.5, 3.67, 3.83, 4.0], [1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 3.25, 3.5, 3.75, 4.0, 4.25, 4.5, 4.75, 5.0, 5.25, 5.5, 5.75, 6.0], [1.75, 2.04, 2.33, 2.62, 2.92, 3.21, 3.5, 3.79, 4.08, 4.38, 4.67, 4.96, 5.25, 5.54, 5.83, 6.12, 6.42, 6.71, 7.0], [1.75, 2.04, 2.33, 2.62, 2.92, 3.21, 3.5, 3.79, 4.08, 4.38, 4.67, 4.96, 5.25, 5.54, 5.83, 6.12, 6.42, 6.71, 7.0]]


In [4]:
def build(n=2, start=0, end=1000):
    
    rs = RandomState(n + start + end)
    size = end - start
    covalent = rs.randint(0, len(cbond_list[0]), size=(size, n))
    hbond = rs.randint(0, len(hbond_list[0]), size=(size, n))
#     hbond = None
#     if n > 1:
#         hbond = rs.randint(0, len(hbondList), size=(size, n-1))
        
    heavy_list = rs.randint(0, len(halogen), size=(size, n))
    
    mols = []
    for i in range(size):
        covalent_inds = covalent[i]
        hbond_inds = hbond[i]
        hbond_inds = None
        if hbond is not None:
            hbond_inds = hbond[i]
            
        mol = qtk.Molecule()
        base_length = 0
        ZR = []
        for j in range(n):
            Zh = halogen[heavy_list[i][j]]
            Hn = base_length
            Fn = base_length + cbond_list[heavy_list[i][j]][covalent_inds[j]]
            ZR.append([ 1, Hn, 0, 0])
            ZR.append([Zh, Fn, 0, 0])
            if n > 1 and j < n - 1:
                base_length += hbond_list[heavy_list[i][j]][hbond_inds[j]]\
                             + cbond_list[heavy_list[i][j]][covalent_inds[j]]
                
        ZR = np.array(ZR)
        
        ZR[:, 1] = ZR[:, 1] - cbond_list[heavy_list[i][0]][covalent_inds[0]]
        mol.build(ZR)
        mol.name = 'HX%d' % n + '_%04d' % i
        mols.append(mol)
    return mols

In [5]:
inps_list = []
for i in range(2,7):
    mols = build(i)
    inps = [qtk.QMInp(m, **qmsetting) for m in mols]
    inps_list.append(inps)

In [6]:
name_list = []
for i in range(2,7):
    name = 'wavelet_HX%d_relaxed_udft' % i
    name_list.append(name)

In [7]:
for i in range(len(inps_list)):
    print name_list[i]
    print inps_list[i][0].molecule.name
    inps_list[i][10].write()

wavelet_HX2_relaxed_udft
HX2_0000
%nproc=1
%chk=tmp.chk
# upbe1pbe/def2tzvp force int(grid=ultrafine) 6d 10f EmpiricalDispersion=GD3 nosymm ExtraLinks=L608 Scf(maxcycle=1000,verytight) IOp(2/12=3)

HX2_0010

0   1
H   -3.2200   0.0000   0.0000
I    0.0000   0.0000   0.0000
H    4.6700   0.0000   0.0000
F    5.5200   0.0000   0.0000


wavelet_HX3_relaxed_udft
HX3_0000
%nproc=1
%chk=tmp.chk
# upbe1pbe/def2tzvp force int(grid=ultrafine) 6d 10f EmpiricalDispersion=GD3 nosymm ExtraLinks=L608 Scf(maxcycle=1000,verytight) IOp(2/12=3)

HX3_0010

0   1
H   -1.0800   0.0000   0.0000
F    0.0000   0.0000   0.0000
H    3.1700   0.0000   0.0000
Br   4.1100   0.0000   0.0000
H    7.9000   0.0000   0.0000
Br  10.2500   0.0000   0.0000


wavelet_HX4_relaxed_udft
HX4_0000
%nproc=1
%chk=tmp.chk
# upbe1pbe/def2tzvp force int(grid=ultrafine) 6d 10f EmpiricalDispersion=GD3 nosymm ExtraLinks=L608 Scf(maxcycle=1000,verytight) IOp(2/12=3)

HX4_0010

0   1
H   -1.8800   0.0000   0.0000
Br   0.0000   0.0000   0

In [8]:
for i in range(len(inps_list)):
    inps = inps_list[i]
    name = name_list[i]
    #qtk.submit(inps, name, **maia)
    #qtk.qmRunAll(inps, name)